# Load Data

In [1]:
import pandas as pd
import json
import progressbar

## Load Reviews
First, load the reviews. We can use chunksize to limit the number of reviews selected.

In [2]:
num_reviews = 2000

In [3]:
df = pd.read_json('data/review.json', lines=True, orient='columns', chunksize=num_reviews)
for chunk in df:
    review = chunk
    break
review.set_index('review_id', inplace=True)
review.head()

,business_id,cool,date,funny,stars,text,useful,user_id
review_id,,,,,,,,
Q1sbwvVQXV2734tPgoKj4Q,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,1,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
GJXCdrto3ASJOqKeVWPi6Q,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,5,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2TzJjDVDEuAW6MR5Vuc1ug,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,5,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
yi0R0Ugj_xUx_Nek0-_Qig,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,5,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
11a8sVPMUFtaC7_ABRkmtw,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,1,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


## Load Supplementary Data
Next, load the corresponding tables. In order to save memory, we only load the data that is referenced in the reviews table.

In [4]:
business_ids = review.business_id.unique()
user_ids = review.user_id.unique()

In [5]:
def load_data(filename, filters, stop_when_done):
    bar = progressbar.ProgressBar(widgets=[progressbar.AnimatedMarker(), " ", progressbar.Counter(), " ", progressbar.BouncingBar(), " ", progressbar.Timer()])
    i = 0
    df_dict = {}
    with open("data/"+filename+".json", encoding='utf-8') as f:
        for line in f:
            obj = json.loads(line)
            add = True
            for col_to_filter, filter_items in filters:
                if (obj[col_to_filter] not in filter_items):
                    add = False
                    break
            if add:
                df_dict[i] = obj
                i+=1
                if stop_when_done and len(df_dict) == len(filter_items):
                    break
            bar.update(len(df_dict))
    bar.finish()
    return pd.DataFrame.from_dict(df_dict, 'index')

In [6]:
business = load_data('business', [('business_id', business_ids)], True)
business.set_index('business_id', inplace=True)
business.head()

| 1469 |       #                                        | Elapsed Time: 0:00:00


,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
business_id,,,,,,,,,,,,,
gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."
fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,16,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ..."
PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119,36.100016,-115.128529,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Restaurants, Italian",None
KWywu2tTEPWmR9JnBc0WyQ,Hunk Mansion,6007 Dean Martin Dr,Las Vegas,NV,89118,36.080168,-115.182756,4.0,107,1,"{'BikeParking': 'False', 'Ambience': '{'romant...","Nightlife, Arts & Entertainment, Bars, Strip C...","{'Thursday': '19:30-2:0', 'Friday': '19:30-3:0..."


In [7]:
user = load_data('user', [('user_id', user_ids)], True)
user.set_index('user_id', inplace=True)
user.head()

| 1959 |                  #                             | Elapsed Time: 0:00:07


,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
user_id,,,,,,,,,,,,,,,,,,,,,
xZAmw5gihOVO4duMN2Ju6Q,Flynn,70,2013-03-25 20:22:45,127,44,28,,"hRbdn-DZAuqYv88bezKrZQ, q3IYh428CVOkU_W90sM3cg...",0,3.21,...,0,0,0,0,2,6,0,0,0,0
yK_WOwDLP8c87hNXVWZBhg,J,10,2010-04-17 19:34:14,7,1,2,,"dQfQok80y8YNwkZCh99Yeg, zeH8s6mOs9b2X5Ae2knhDw...",1,3.42,...,0,0,0,0,3,0,0,0,0,0
yMh4OF_Li2gJzE5rJ8oCbA,Mike,531,2010-08-16 01:47:53,1315,923,982,"2012,2013,2014,2015,2016,2017,2018","JO8EVwN_AwYxhieTOgnXyQ, kJKiCzXG7TSsq-91XCPUWA...",45,3.92,...,4,0,0,1,44,49,88,88,49,10
lsLuVKmdh45yommr53zfRg,Penny,70,2014-05-21 23:54:26,60,12,22,2017,"ewkj5-77i6LVGo-5rqIdBw, 9gIfIWeSEkX2pfza7wJmag...",1,3.65,...,0,0,0,0,0,0,1,1,1,0
UG4EKu13JRwzRix6ESINdg,Edward,1083,2009-06-05 16:32:55,2365,1013,1315,"2011,2012,2013,2014,2015,2016,2017,2018","jZbVe9Imk2WZrhLtkrWdaQ, BOUsgSa-XWcb3s8-NqEu8A...",53,3.61,...,13,2,1,0,54,117,93,93,18,4


In [8]:
checkin = load_data('checkin', [('business_id', business_ids)], False)
checkin.set_index('business_id', inplace=True)
checkin.head()

| 1403 |                                   #            | Elapsed Time: 0:00:06


,date
business_id,
-000aQFeK6tqVLndf7xORg,2018-10-17 21:16:27
-1xuC540Nycht_iWFeJ-dw,"2010-04-25 12:27:32, 2010-06-15 16:47:42, 2010..."
-4TMQnQJW1yd6NqGRDvAeA,"2012-09-16 02:04:06, 2012-09-23 03:33:28, 2012..."
-BJVR_DO5r-MfQ2tiszeOg,"2011-06-12 06:02:21, 2011-06-21 22:09:11, 2012..."
-DHKF52ALa4B2ls1V6OlxA,"2016-03-04 18:25:01, 2018-10-22 15:39:42"


In [9]:
photo = load_data('photo', [('business_id', business_ids)], False)
photo.set_index('business_id', inplace=True)
photo.head()

| 7301 |                              #                 | Elapsed Time: 0:00:06


,caption,photo_id,label
business_id,,,
296PZdxSrtH08EUwCsOKMw,,1eDvPC4F8cGvuI2lGpIoEw,inside
7eQoxWr9RzyCB9IFvqHUPA,,GuvDS21yJ5efL1Zo1EzGPA,outside
GCRvrxMSC1nzShyM4Y-guQ,Bar Dancers,hkV_CrgjTeJBTAWJWAO46w,inside
4hG2j_ibsNblDgqei05U_g,"Best Ribs on the Planet. OK, so I took the ph...",UJkd_PoWNr18Yqhsb2jELA,food
4hG2j_ibsNblDgqei05U_g,,AYHMdVOF7juF13QlabzZUw,inside


In [10]:
# tip = load_data('tip', [('business_id', business_ids), ('user_id', user_ids)], False)
# tip.head()

In [11]:
import gc
gc.collect()

7

## Export Data

We export the data for easier loading for future sessions. 

In [ ]:
review.to_parquet("data/"+str(num_reviews)+"_review.parquet", engine='fastparquet', compression='gzip')
business.to_parquet("data/"+str(num_reviews)+"_business.parquet", engine='fastparquet', compression='gzip')
user.to_parquet("data/"+str(num_reviews)+"_user.parquet", engine='fastparquet', compression='gzip')
checkin.to_parquet("data/"+str(num_reviews)+"_checkin.parquet", engine='fastparquet', compression='gzip')
photo.to_parquet("data/"+str(num_reviews)+"_photo.parquet", engine='fastparquet', compression='gzip')